In [59]:
import pandas as pd
import numpy as np
import gensim
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import accuracy_score
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\upend\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [91]:
# Load your dataset (assuming it's in a pandas DataFrame)
df = pd.read_csv('Twitter_ Non-Advert-Tabelle 1.csv',delimiter=';')

# Preprocess text data
stemmer = PorterStemmer()
english_stopwords = set(stopwords.words("english"))
def preprocess_text(text):
    # Add your preprocessing steps here (e.g., removing stopwords, punctuation, etc.)
    # Convert text to lowercase
    text = str(text).lower()
    
    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # Remove digits and single characters
    text = re.sub(r'\b\w\b|\d+', '', text)
    
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    
    # Tokenize the text and remove stopwords
    words = [word for word in text.split() if word not in english_stopwords]
    
    # Stem the words
    stemmed_words = [stemmer.stem(word) for word in words]
    
    # Join the stemmed words back into a single string
    processed_text = ' '.join(stemmed_words)
    
    return processed_text

# Apply preprocessing to the text column
df['text'] = df['text'].apply(preprocess_text)

# Split data into features (X) and labels (y)
X = df['text']
y = df['label']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [92]:
# Tokenize text data
tokenized_text = [gensim.utils.simple_preprocess(text) for text in X_train]

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_text, vector_size=100, window=5, min_count=1, workers=4)


In [93]:
import numpy as np

# Define a function to calculate the document vector
def document_vector(word2vec_model, doc):
    # Filter out words not in the vocabulary of the Word2Vec model
    doc = [word for word in doc if word in word2vec_model.wv.key_to_index]
    # Get vectors for each word in the document and calculate the mean
    vectors = [word2vec_model.wv.get_vector(word) for word in doc]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)  # Return zero vector if no words are found in the model

# Vectorize train and test data
X_train_word2vec = np.array([document_vector(word2vec_model, doc) for doc in tokenized_text])
X_test_word2vec = np.array([document_vector(word2vec_model, doc) for doc in [gensim.utils.simple_preprocess(text) for text in X_test]])


In [94]:
# Initialize and train the classifier
lr_clf = LogisticRegression(max_iter=200)
lr_clf.fit(X_train_word2vec, y_train)


LogisticRegression(max_iter=200)

In [95]:
# Predict on test data
y_pred = lr_clf.predict(X_test_word2vec)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.6690997566909975


In [106]:
def document_vector(word2vec_model, doc):
    doc = [word for word in doc if word in word2vec_model.wv.key_to_index]
    vectors = [word2vec_model.wv.get_vector(word) for word in doc]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

# Preprocess and tokenize the input text
input_text = "i want to live happily"
tokenized_text = gensim.utils.simple_preprocess(input_text)
print(tokenized_text)
# Calculate the document vector for the input text
input_vector = document_vector(word2vec_model, tokenized_text)
print(input_vector)
# Make prediction using your classifier (assuming you have trained one)
# Replace `your_classifier` with your actual classifier instance
prediction = lr_clf.predict([input_vector])

# Print the prediction
print(prediction)

['want', 'to', 'live', 'happily']
[-0.03029973  0.04760795  0.0129469  -0.01356089  0.01101695 -0.07141391
  0.0145027   0.10743617 -0.03073135 -0.02356633 -0.00982142 -0.05644203
 -0.0160173   0.02832618  0.02161339 -0.03794704 -0.00482025 -0.05425112
  0.00471125 -0.07949834  0.03214223  0.01728191  0.03315314 -0.02511072
 -0.00546616 -0.00701783 -0.02009497 -0.03546325 -0.04502528 -0.0199047
  0.04345102  0.02193569  0.01206699 -0.05326926  0.00713709  0.02763154
 -0.00728782 -0.02237236 -0.02641438 -0.07423244 -0.00417077 -0.03512901
 -0.02270683  0.00753505  0.02315339 -0.02221668 -0.0389581   0.01688055
  0.02941999  0.05114432  0.0093892  -0.04024952 -0.01753298 -0.01364054
 -0.00058104  0.00914877  0.03636127 -0.01096748 -0.027807    0.01659053
  0.01348197  0.0360746  -0.03221151  0.00412225 -0.06563617  0.04536305
 -0.00242074  0.03591803 -0.06619024  0.05752125 -0.03168175  0.02452806
  0.0563981  -0.02032609  0.05247672  0.00918039  0.0115796  -0.0159373
 -0.04580217  0.009